In [1]:
from pymongo import MongoClient
import json
import pandas as pd
import numpy as np
import re


client = MongoClient('mongodb://localhost:27017/')
db = client.companies

In [2]:
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'companies')

## Vamos a hacer un listado con las empresas que no cumplen con nuestros requisitos para tenerlas ubicadas y así poder establecer descartes en base a la cercanía a estas empresas.

In [3]:
companies_dislike = db.companies.find({'$and':
                                      [{'deadpooled_year':{'$eq':None}},{'founded_year':{'$lt':2008}},
                                    {"offices.latitude":{'$exists':True}},{"offices.longitude":{'$exists':True}},
                                    {"offices.latitude":{'$ne':None}}, {"offices.longitude":{'$ne':None}}]})


#### Convertimos el objeto pymongo en DataFrame

In [4]:
companies_dislike = pd.DataFrame(list(companies_dislike))
companies_dislike.head(2)

,_id,acquisition,acquisitions,alias_list,blog_feed_url,blog_url,category_code,competitions,created_at,crunchbase_url,...,phone_number,products,providerships,relationships,screenshots,tag_list,total_money_raised,twitter_username,updated_at,video_embeds
0,52cdef7c4bab8bd675297d8d,"{'acquired_year': 2012, 'acquired_day': 12, 'p...","[{'acquired_year': 2009, 'acquired_day': 15, '...",,http://blog.digg.com/?feed=rss2,http://blog.digg.com/,news,"[{'competitor': {'name': 'Reddit', 'permalink'...",Fri May 25 20:03:23 UTC 2007,http://www.crunchbase.com/company/digg,...,(415) 436-9638,"[{'name': 'Digg', 'permalink': 'digg'}]","[{'title': 'Public Relations', 'provider': {'n...","[{'title': 'CEO', 'person': {'permalink': 'and...","[{'available_sizes': [[[117, 150], 'assets/ima...","community, social, news, bookmark, digg, techn...",$45M,digg,Tue Nov 05 21:35:47 UTC 2013,"[{'embed_code': '<embed src=""http://blip.tv/pl..."
1,52cdef7c4bab8bd675297d90,"{'acquired_year': 2007, 'acquired_day': 9, 'pr...",[],None,,,web,"[{'competitor': {'name': 'RescueTime', 'permal...",Fri Jun 08 12:19:51 UTC 2007,http://www.crunchbase.com/company/postini,...,888.584.3150,"[{'name': 'Postini Solutions', 'permalink': 'p...",[],"[{'title': 'Board', 'person': {'permalink': 's...",[],,$0,None,Sat Aug 13 18:02:34 UTC 2011,[]


#### Establecemos el index con el ID de mongodb

In [5]:
companies_dislike.set_index('_id', inplace=True)
companies_dislike.head(2)

,acquisition,acquisitions,alias_list,blog_feed_url,blog_url,category_code,competitions,created_at,crunchbase_url,deadpooled_day,...,phone_number,products,providerships,relationships,screenshots,tag_list,total_money_raised,twitter_username,updated_at,video_embeds
_id,,,,,,,,,,,,,,,,,,,,,
52cdef7c4bab8bd675297d8d,"{'acquired_year': 2012, 'acquired_day': 12, 'p...","[{'acquired_year': 2009, 'acquired_day': 15, '...",,http://blog.digg.com/?feed=rss2,http://blog.digg.com/,news,"[{'competitor': {'name': 'Reddit', 'permalink'...",Fri May 25 20:03:23 UTC 2007,http://www.crunchbase.com/company/digg,None,...,(415) 436-9638,"[{'name': 'Digg', 'permalink': 'digg'}]","[{'title': 'Public Relations', 'provider': {'n...","[{'title': 'CEO', 'person': {'permalink': 'and...","[{'available_sizes': [[[117, 150], 'assets/ima...","community, social, news, bookmark, digg, techn...",$45M,digg,Tue Nov 05 21:35:47 UTC 2013,"[{'embed_code': '<embed src=""http://blip.tv/pl..."
52cdef7c4bab8bd675297d90,"{'acquired_year': 2007, 'acquired_day': 9, 'pr...",[],None,,,web,"[{'competitor': {'name': 'RescueTime', 'permal...",Fri Jun 08 12:19:51 UTC 2007,http://www.crunchbase.com/company/postini,None,...,888.584.3150,"[{'name': 'Postini Solutions', 'permalink': 'p...",[],"[{'title': 'Board', 'person': {'permalink': 's...",[],,$0,None,Sat Aug 13 18:02:34 UTC 2011,[]


##### Eliminamos las columnas que no nos son de utilidad

In [6]:
companies_dislike.drop(companies_dislike.columns[[0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 14, 15, 16, 17, 20, 21, 22, 24, 28, 29, 30, 31, 32, 33, 34, 35, 38, 39, 40
]], axis=1, inplace=True)

companies_dislike.head(2)

,category_code,deadpooled_year,description,founded_year,funding_rounds,ipo,name,number_of_employees,offices,tag_list,total_money_raised
_id,,,,,,,,,,,
52cdef7c4bab8bd675297d8d,news,None,user driven social content website,2004,"[{'funded_month': 12, 'investments': [{'compan...",None,Digg,60.0,"[{'address2': '', 'country_code': 'USA', 'desc...","community, social, news, bookmark, digg, techn...",$45M
52cdef7c4bab8bd675297d90,web,None,None,1999,[],None,Postini,NaN,"[{'address2': None, 'country_code': 'USA', 'de...",,$0


### Ahora vamos a establecer la geolocalizacion, para luego ponerla en el geoindex

In [7]:
def getFirst(data):
    data = data['offices']

    principal = None
    if data[0]['latitude'] and data[0]['longitude']:
        principal = {
            "type":"Point",
            "coordinates":[data[0]['longitude'], data[0]['latitude']]
        }

    return {
        "totalOffices": len(data),
        "lat": data[0]['latitude'],
        "lng": data[0]['longitude'],
        "oficina_principal": principal
    }


comp_dislike_off = companies_dislike[["offices"]].apply(getFirst, result_type="expand", axis=1)

##### Incluimos en el DataFrame esta información

In [8]:
companies_dislike = pd.concat([companies_dislike, comp_dislike_off], axis=1)
companies_dislike.head(2)

,category_code,deadpooled_year,description,founded_year,funding_rounds,ipo,name,number_of_employees,offices,tag_list,total_money_raised,lat,lng,oficina_principal,totalOffices
_id,,,,,,,,,,,,,,,
52cdef7c4bab8bd675297d8d,news,None,user driven social content website,2004,"[{'funded_month': 12, 'investments': [{'compan...",None,Digg,60.0,"[{'address2': '', 'country_code': 'USA', 'desc...","community, social, news, bookmark, digg, techn...",$45M,37.764726,-122.394523,"{'type': 'Point', 'coordinates': [-122.394523,...",1
52cdef7c4bab8bd675297d90,web,None,None,1999,[],None,Postini,NaN,"[{'address2': None, 'country_code': 'USA', 'de...",,$0,37.506885,-122.247573,"{'type': 'Point', 'coordinates': [-122.247573,...",1


##### Esta lista la ultilizaré en el siguiente paso, ya que no veo la forma de establecer áreas negras para realizar descartes. En su lugar, al obtener la lista de empresas cercanas, voy a descartar esa empresa si una de estas empresas aparece en el listado

In [9]:
names_companies_dislike = list(companies_dislike['name'])

###### Guardamos el dataframe en un json

In [10]:
companies_dislike.to_json('./companies_dislike.json', orient='records', default_handler=str)

#### Hacemos un filtro de las empresas que pueden encajar con nuestros requisitos

In [11]:
filtered_companies = db.companies.find({'$and': 
                                        [{'deadpooled_year':{'$eq':None}}, 
                                         {'founded_year':{'$gte':2008}},
                                         {'total_money_raised': {'$ne': "$0"}},
                                         {'total_money_raised': {'$ne': None}},
                                         {"offices.latitude":{'$exists':True}},
                                         {"offices.longitude":{'$exists':True}},
                                         {"offices.latitude":{'$ne':None}},
                                         {"offices.longitude":{'$ne':None}},
                                         {'category_code':{'$not':{'$eq':None}}}, 
                                         {'category_code':{'$not':{'$eq':'education'}}},
                                         {'category_code':{'$not':{'$eq':'fashion'}}}, 
                                         {'category_code':{'$not':{'$eq':'health'}}},
                                         {'category_code':{'$not':{'$eq':'hospitality'}}}, 
                                         {'category_code':{'$not':{'$eq':'legal'}}},
                                         {'category_code':{'$not':{'$eq':'medical'}}}, 
                                         {'category_code':{'$not':{'$eq':'other'}}},
                                         {'category_code':{'$not':{'$eq':'real_estate'}}}, 
                                         {'category_code':{'$not':{'$eq':'travel'}}}
                                        ]})


#### Convertimos el objeto pymongo en DataFrame

In [12]:
filtered_companies = pd.DataFrame(list(filtered_companies))
print(filtered_companies.shape)
filtered_companies.head(2)


(319, 42)


,_id,acquisition,acquisitions,alias_list,blog_feed_url,blog_url,category_code,competitions,created_at,crunchbase_url,...,phone_number,products,providerships,relationships,screenshots,tag_list,total_money_raised,twitter_username,updated_at,video_embeds
0,52cdef7c4bab8bd675297e30,None,[],None,,http://goingon.com/blog,software,"[{'competitor': {'name': 'Ning by Glam Media',...",Fri Jul 20 04:55:26 UTC 2007,http://www.crunchbase.com/company/goingon,...,415-200-4606,"[{'name': 'GoingOn', 'permalink': 'goingon'}, ...",[],"[{'title': 'CEO', 'person': {'permalink': 'jon...",[],"socialnetworking, enterprise, social-media, so...",$8.5M,GoingOn_Network,Thu Apr 04 09:03:17 UTC 2013,[]
1,52cdef7c4bab8bd675297ee1,None,[],,,http://blog.cloudant.com/,enterprise,[],Fri Aug 17 12:40:06 UTC 2007,http://www.crunchbase.com/company/cloudant,...,(857) 400-9900,[],[],"[{'title': 'Founder & President', 'person': {'...",[],"cloud-computing, big-data, databases, software...",$18.3M,cloudant,Mon Nov 11 19:25:28 UTC 2013,"[{'embed_code': '<iframe width=""430"" height=""2..."


##### Establecemos el index con el ID de mongodb

In [13]:
filtered_companies.set_index('_id', inplace=True)
filtered_companies.head(2)

,acquisition,acquisitions,alias_list,blog_feed_url,blog_url,category_code,competitions,created_at,crunchbase_url,deadpooled_day,...,phone_number,products,providerships,relationships,screenshots,tag_list,total_money_raised,twitter_username,updated_at,video_embeds
_id,,,,,,,,,,,,,,,,,,,,,
52cdef7c4bab8bd675297e30,None,[],None,,http://goingon.com/blog,software,"[{'competitor': {'name': 'Ning by Glam Media',...",Fri Jul 20 04:55:26 UTC 2007,http://www.crunchbase.com/company/goingon,None,...,415-200-4606,"[{'name': 'GoingOn', 'permalink': 'goingon'}, ...",[],"[{'title': 'CEO', 'person': {'permalink': 'jon...",[],"socialnetworking, enterprise, social-media, so...",$8.5M,GoingOn_Network,Thu Apr 04 09:03:17 UTC 2013,[]
52cdef7c4bab8bd675297ee1,None,[],,,http://blog.cloudant.com/,enterprise,[],Fri Aug 17 12:40:06 UTC 2007,http://www.crunchbase.com/company/cloudant,None,...,(857) 400-9900,[],[],"[{'title': 'Founder & President', 'person': {'...",[],"cloud-computing, big-data, databases, software...",$18.3M,cloudant,Mon Nov 11 19:25:28 UTC 2013,"[{'embed_code': '<iframe width=""430"" height=""2..."


#### Eliminamos las columnas que no nos son de utilidad

In [14]:
filtered_companies.drop(filtered_companies.columns[[0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 14, 15, 16, 17, 20, 21, 22, 24, 28, 29, 30, 31, 32, 33, 34, 35, 38, 39, 40
]], axis=1, inplace=True)

filtered_companies.head(2)

,category_code,deadpooled_year,description,founded_year,funding_rounds,ipo,name,number_of_employees,offices,tag_list,total_money_raised
_id,,,,,,,,,,,
52cdef7c4bab8bd675297e30,software,None,Private Academic Networks,2008,"[{'funded_month': 8, 'investments': [{'company...",None,GoingOn,40.0,"[{'address2': '12th Floor', 'country_code': 'U...","socialnetworking, enterprise, social-media, so...",$8.5M
52cdef7c4bab8bd675297ee1,enterprise,None,NoSQL database managed service,2008,"[{'funded_month': 12, 'investments': [{'compan...",None,Cloudant,70.0,"[{'address2': '', 'country_code': 'USA', 'desc...","cloud-computing, big-data, databases, software...",$18.3M


#### Ahora vamos a establecer la geolocalizacion, para luego ponerla en el geoindex

In [15]:
fil_comp_off = filtered_companies[["offices"]].apply(getFirst, result_type="expand", axis=1)

##### Incluimos en el DataFrame esta información

In [16]:
filtered_companies = pd.concat([filtered_companies, fil_comp_off], axis=1)
filtered_companies.head(2)

,category_code,deadpooled_year,description,founded_year,funding_rounds,ipo,name,number_of_employees,offices,tag_list,total_money_raised,lat,lng,oficina_principal,totalOffices
_id,,,,,,,,,,,,,,,
52cdef7c4bab8bd675297e30,software,None,Private Academic Networks,2008,"[{'funded_month': 8, 'investments': [{'company...",None,GoingOn,40.0,"[{'address2': '12th Floor', 'country_code': 'U...","socialnetworking, enterprise, social-media, so...",$8.5M,37.782263,-122.392142,"{'type': 'Point', 'coordinates': [-122.392142,...",1
52cdef7c4bab8bd675297ee1,enterprise,None,NoSQL database managed service,2008,"[{'funded_month': 12, 'investments': [{'compan...",None,Cloudant,70.0,"[{'address2': '', 'country_code': 'USA', 'desc...","cloud-computing, big-data, databases, software...",$18.3M,42.358920,-71.057810,"{'type': 'Point', 'coordinates': [-71.05781, 4...",1


# Ahora vamos a normalizar la columna 'total_money_raised'

In [17]:
lista = list(filtered_companies['total_money_raised'])

In [18]:
# tipos moneda --> $, £, €, C$ (empiezan por este valor y terminan por k, M ó B) - C$ = dolar canadiense 


# Exchange rates from https://www.federalreserve.gov/releases/g5a/current/


# 2018 Exchange rates --> C$ = 1.2957 - € = 1.1817 - £ = 1.3363 


In [19]:
def exchangeMoney (value):
    ex_rate = {'C$':1.2957, '€':1.1817, '£':1.3363}
    amount = {'k':1000, 'M':1000000, 'B': 1000000000}
    value_finder = float(re.findall('[0-9]*\.[0-9]|[0-9]\d*',value)[0])
    abb = ('k','K')
    
    if value.endswith(abb) & value.startswith("$"):
        r_amt = int(value_finder * (amount['k']))    
    elif value.endswith(abb) & value.startswith("C$"):
        r_amt = int(value_finder * (amount['k']) * (ex_rate['C$']))    
    elif value.endswith(abb) & value.startswith("€"):
        r_amt = int(value_finder * (amount['k']) * (ex_rate['€']))      
    elif value.endswith(abb) & value.startswith("£"):
        r_amt = int(value_finder * (amount['k']) * (ex_rate['£']))
                
    elif value.endswith('M') & value.startswith("$"):
        r_amt = int(value_finder * (amount['M']))             
    elif value.endswith('M') & value.startswith("C$"):
        r_amt = int(value_finder * (amount['M']) * (ex_rate['C$']))    
    elif value.endswith('M') & value.startswith("€"):
        r_amt = int(value_finder * (amount['M']) * (ex_rate['€']))    
    elif value.endswith('M') & value.startswith("£"):
        r_amt = int(value_finder * (amount['M']) * (ex_rate['£'])) 
    
    elif value.endswith('B') & value.startswith("$"):
        r_amt = int(value_finder*(amount['B']))
    elif value.endswith('B') & value.startswith("C$"):
        r_amt = int(value_finder*(amount['B'])*(ex_rate['C$']))   
    elif value.endswith('B') & value.startswith("€"):
        r_amt = int(value_finder*(amount['B'])*(ex_rate['€']))    
    elif value.endswith('B') & value.startswith("£"):
        r_amt = int(value_finder*(amount['B'])*(ex_rate['£'])) 

    return r_amt

In [20]:
filtered_companies['total_money_raised_USD'] = filtered_companies['total_money_raised'].apply(exchangeMoney)

In [21]:
filtered_companies.head(3)

,category_code,deadpooled_year,description,founded_year,funding_rounds,ipo,name,number_of_employees,offices,tag_list,total_money_raised,lat,lng,oficina_principal,totalOffices,total_money_raised_USD
_id,,,,,,,,,,,,,,,,
52cdef7c4bab8bd675297e30,software,None,Private Academic Networks,2008,"[{'funded_month': 8, 'investments': [{'company...",None,GoingOn,40.0,"[{'address2': '12th Floor', 'country_code': 'U...","socialnetworking, enterprise, social-media, so...",$8.5M,37.782263,-122.392142,"{'type': 'Point', 'coordinates': [-122.392142,...",1,8500000
52cdef7c4bab8bd675297ee1,enterprise,None,NoSQL database managed service,2008,"[{'funded_month': 12, 'investments': [{'compan...",None,Cloudant,70.0,"[{'address2': '', 'country_code': 'USA', 'desc...","cloud-computing, big-data, databases, software...",$18.3M,42.358920,-71.057810,"{'type': 'Point', 'coordinates': [-71.05781, 4...",1,18300000
52cdef7c4bab8bd675297f76,games_video,None,Digital Entertainment and Media Platform,2008,"[{'funded_month': 2, 'investments': [{'company...",None,Crunchyroll,50.0,"[{'address2': '', 'country_code': 'USA', 'desc...","crunchyroll, anime, manga, video, gaming, musi...",$4.8M,37.781265,-122.393229,"{'type': 'Point', 'coordinates': [-122.393229,...",1,4800000


#### Ahora vamos a filtrar por las empresas que han facturado más de un millon

In [22]:
higher_1M = filtered_companies['total_money_raised_USD'] >= 1000000
filtered_companies2 = filtered_companies[higher_1M]
print(filtered_companies2.shape)
filtered_companies2.head(3)

(223, 16)


,category_code,deadpooled_year,description,founded_year,funding_rounds,ipo,name,number_of_employees,offices,tag_list,total_money_raised,lat,lng,oficina_principal,totalOffices,total_money_raised_USD
_id,,,,,,,,,,,,,,,,
52cdef7c4bab8bd675297e30,software,None,Private Academic Networks,2008,"[{'funded_month': 8, 'investments': [{'company...",None,GoingOn,40.0,"[{'address2': '12th Floor', 'country_code': 'U...","socialnetworking, enterprise, social-media, so...",$8.5M,37.782263,-122.392142,"{'type': 'Point', 'coordinates': [-122.392142,...",1,8500000
52cdef7c4bab8bd675297ee1,enterprise,None,NoSQL database managed service,2008,"[{'funded_month': 12, 'investments': [{'compan...",None,Cloudant,70.0,"[{'address2': '', 'country_code': 'USA', 'desc...","cloud-computing, big-data, databases, software...",$18.3M,42.358920,-71.057810,"{'type': 'Point', 'coordinates': [-71.05781, 4...",1,18300000
52cdef7c4bab8bd675297f76,games_video,None,Digital Entertainment and Media Platform,2008,"[{'funded_month': 2, 'investments': [{'company...",None,Crunchyroll,50.0,"[{'address2': '', 'country_code': 'USA', 'desc...","crunchyroll, anime, manga, video, gaming, musi...",$4.8M,37.781265,-122.393229,"{'type': 'Point', 'coordinates': [-122.393229,...",1,4800000


In [23]:
"""
Códigos de terminal para importar a mongodb las bases de datos y así poder establecer el geoindex
---------------------------------------------------------------


mongoimport --db companies --collection companies_dislike --file companies_dislike.json --jsonArray

mongoimport --db companies --collection filtered_companies --file filtered_companies.json --jsonArray

"""

'\nCódigos de terminal para importar a mongodb las bases de datos y así poder establecer el geoindex\n---------------------------------------------------------------\n\n\nmongoimport --db companies --collection companies_dislike --file companies_dislike.json --jsonArray\n\nmongoimport --db companies --collection filtered_companies --file filtered_companies.json --jsonArray\n\n'

In [24]:
filtered_companies2.to_json('./filtered_companies.json', orient='records', default_handler=str)